In [1]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

from repeng import ControlVector, ControlModel, DatasetEntry

# load and wrap Mistral-7B
model_name = "Qwen/Qwen2.5-7B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = ControlModel(model, list(range(-1, -27, -1)))

def make_dataset(base_steps, next_steps):
    dataset = []
    for base, next in zip(base_steps, next_steps):
        dataset.append(
            DatasetEntry(
                positive=next,
                negative=base,
            )
        )
    return dataset


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
nextsteps = ["Solving the following mathematical problem. Problem: Calculate the following expression: (4 + 3 * 2 - 1 ) * 4 - 2. Step 1: First, we add 4 and 3 to get 7.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 23 * 4 - 1 ) * 3 - 2. Step 1: First, we add 12 and 23 to get 35.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (34 + 2231 * 6 - 1 ) * 34 - 2. Step 1: First, we add 34 and 2231 to get 2265.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 123 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 123 to get 135.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12445 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 12445 to get 12457.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12446 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 123 to get 12458.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12447 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 123 to get 12459.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 1000 * 2 - 1 ) * 412 - 2. Step 1: First, we add 12 and 1000 to get 1012.",]


basesteps = ["Solving the following mathematical problem. Problem: Calculate the following expression: (4 + 3 * 2 - 1 ) * 4 - 3111.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 23 * 4 - 1 ) * 3 - 1245.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (34 + 2231 * 6 - 1 ) * 34 - 123.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 123 * 2 - 1 ) * 412 - 123.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12445 * 2 - 1 ) * 412 - 2.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12446 * 2 - 1 ) * 412 - 2.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 12447 * 2 - 1 ) * 412 - 2.",
             "Solving the following mathematical problem. Problem: Calculate the following expression: (12 + 1000 * 2 - 1 ) * 412 - 2.",]

query = "Solving the following mathematical problem. Problem: Calculate the following expression: (10222 + 23123123 * 4 - 1 ) * 5 - 6."

In [3]:
experience_dataset = make_dataset(basesteps, nextsteps)
# train the vector—takes less than a minute!
exp_vec = ControlVector.train(model, tokenizer, experience_dataset)

# set the control strength and let inference rip!
for strength in (-2.2, 1, 1.5, 2.2, 2.5, 3, 3.5):
    print(f"strength={strength}")
    model.set_control(exp_vec, strength)
    out = model.generate(
        **{key: value.cuda() for key, value in tokenizer(
            query,
            return_tensors="pt"
        ).items()},
        do_sample=False,
        max_new_tokens=512,
        repetition_penalty=1.1,
    )
    print(tokenizer.decode(out.squeeze()).strip())
    print()

100%|██████████| 27/27 [00:00<00:00, 245.99it/s]
/datadrive5/huypn16/anaconda3/envs/rloo/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/datadrive5/huypn16/anaconda3/envs/rloo/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:606: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/datadrive5/huypn16/anaconda3/envs/rloo/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:623: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` 

strength=-2.2
Solving the following mathematical problem. Problem: Calculate the following expression: (10222 + 23123123 * 4 - 1 ) * 5 - 6. What is the result of this calculation? The order of operations in mathematics, often remembered by the acronym PEMDAS (Parentheses, Exponents, Multiplication and Division (from left to right), Addition and Subtraction (from left to right)), should be followed when solving this expression.

So we first perform multiplication:

(23123123 * 4) = 92492492

Then we do addition and subtraction from left to right:

(10222 + 92492492 - 1) = 92493713

Finally, we multiply the result by 5 and then subtract 6:

(92493713 * 5) - 6 = 46246856

Therefore, the answer is 46246856. 

Please note that there might be a typo in your input as '23123123' seems like an unusually large number for such a simple arithmetic operation. If you meant something else, please clarify. Otherwise, based on the given numbers, the solution is 46246856. Please let me know if I can ass